In [ ]:
from keras.layers.convolutional import Conv2D
from keras.layers import Dense, Flatten
from keras.optimizers import RMSprop
from keras.models import Sequential
from skimage.transform import resize
from skimage.color import rgb2gray
from collections import deque
from keras import backend as K
import tensorflow as tf
import numpy as np
import random
import gym

EPISODES = 1000


# 브레이크아웃에서의 DQN 에이전트
class DQNAgent:
    def __init__(self, action_size):
        self.render = False
        self.load_model = False
        # 상태와 행동의 크기 정의
        self.state_size = (84, 84, 4)
        self.action_size = action_size
        # DQN 하이퍼파라미터
        self.epsilon = 1.
        self.epsilon_start, self.epsilon_end = 1.0, 0.1
        self.exploration_steps = 1000000.
        self.epsilon_decay_step = (self.epsilon_start - self.epsilon_end) \
                                  / self.exploration_steps
        self.batch_size = 32
        self.train_start = 50000
        self.update_target_rate = 10000
        self.discount_factor = 0.99
        # 리플레이 메모리, 최대 크기 400000
        self.memory = deque(maxlen=400000)
        self.no_op_steps = 30
        # 모델과 타겟모델을 생성하고 타겟모델 초기화
        self.model = self.build_model()
        self.target_model = self.build_model()
        self.update_target_model()

        self.optimizer = self.optimizer()

        # 텐서보드 설정
        self.sess = tf.InteractiveSession()
        K.set_session(self.sess)

        self.avg_q_max, self.avg_loss = 0, 0
        self.summary_placeholders, self.update_ops, self.summary_op = \
            self.setup_summary()
        self.summary_writer = tf.summary.FileWriter(
            'summary/breakout_dqn', self.sess.graph)
        self.sess.run(tf.global_variables_initializer())

        if self.load_model:
            self.model.load_weights("./save_model/breakout_dqn.h5")

    # Huber Loss를 이용하기 위해 최적화 함수를 직접 정의
    def optimizer(self):
        a = K.placeholder(shape=(None,), dtype='int32')
        y = K.placeholder(shape=(None,), dtype='float32')

        prediction = self.model.output

        a_one_hot = K.one_hot(a, self.action_size)
        q_value = K.sum(prediction * a_one_hot, axis=1)
        error = K.abs(y - q_value)

        quadratic_part = K.clip(error, 0.0, 1.0)
        linear_part = error - quadratic_part
        loss = K.mean(0.5 * K.square(quadratic_part) + linear_part)

        optimizer = RMSprop(lr=0.00025, epsilon=0.01)
        updates = optimizer.get_updates(self.model.trainable_weights, [], loss)
        train = K.function([self.model.input, a, y], [loss], updates=updates)

        return train

    # 상태가 입력, 큐함수가 출력인 인공신경망 생성
    def build_model(self):
        model = Sequential()
        model.add(Conv2D(32, (8, 8), strides=(4, 4), activation='relu',
                         input_shape=self.state_size))
        model.add(Conv2D(64, (4, 4), strides=(2, 2), activation='relu'))
        model.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu'))
        model.add(Flatten())
        model.add(Dense(512, activation='relu'))
        model.add(Dense(self.action_size))
        model.summary()
        return model

    # 타겟 모델을 모델의 가중치로 업데이트
    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    # 입실론 탐욕 정책으로 행동 선택
    def get_action(self, history):
        history = np.float32(history / 255.0)
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        else:
            q_value = self.model.predict(history)
            return np.argmax(q_value[0])

    # 샘플 <s, a, r, s'>을 리플레이 메모리에 저장
    def append_sample(self, history, action, reward, next_history, dead):
        self.memory.append((history, action, reward, next_history, dead))

    # 리플레이 메모리에서 무작위로 추출한 배치로 모델 학습
    def train_model(self):
        if self.epsilon > self.epsilon_end:
            self.epsilon -= self.epsilon_decay_step

        mini_batch = random.sample(self.memory, self.batch_size)

        history = np.zeros((self.batch_size, self.state_size[0],
                            self.state_size[1], self.state_size[2]))
        next_history = np.zeros((self.batch_size, self.state_size[0],
                                 self.state_size[1], self.state_size[2]))
        target = np.zeros((self.batch_size,))
        action, reward, dead = [], [], []

        for i in range(self.batch_size):
            history[i] = np.float32(mini_batch[i][0] / 255.)
            next_history[i] = np.float32(mini_batch[i][3] / 255.)
            action.append(mini_batch[i][1])
            reward.append(mini_batch[i][2])
            dead.append(mini_batch[i][4])

        target_value = self.target_model.predict(next_history)

        for i in range(self.batch_size):
            if dead[i]:
                target[i] = reward[i]
            else:
                target[i] = reward[i] + self.discount_factor * \
                                        np.amax(target_value[i])

        loss = self.optimizer([history, action, target])
        self.avg_loss += loss[0]

    # 각 에피소드 당 학습 정보를 기록
    def setup_summary(self):
        episode_total_reward = tf.Variable(0.)
        episode_avg_max_q = tf.Variable(0.)
        episode_duration = tf.Variable(0.)
        episode_avg_loss = tf.Variable(0.)

        tf.summary.scalar('Total Reward/Episode', episode_total_reward)
        tf.summary.scalar('Average Max Q/Episode', episode_avg_max_q)
        tf.summary.scalar('Duration/Episode', episode_duration)
        tf.summary.scalar('Average Loss/Episode', episode_avg_loss)

        summary_vars = [episode_total_reward, episode_avg_max_q,
                        episode_duration, episode_avg_loss]
        summary_placeholders = [tf.placeholder(tf.float32) for _ in
                                range(len(summary_vars))]
        update_ops = [summary_vars[i].assign(summary_placeholders[i]) for i in
                      range(len(summary_vars))]
        summary_op = tf.summary.merge_all()
        return summary_placeholders, update_ops, summary_op


# 학습속도를 높이기 위해 흑백화면으로 전처리
def pre_processing(observe):
    processed_observe = np.uint8(
        resize(rgb2gray(observe), (84, 84), mode='constant') * 255)
    return processed_observe


if __name__ == "__main__":
    # 환경과 DQN 에이전트 생성
    env = gym.make('BreakoutDeterministic-v4')
    agent = DQNAgent(action_size=3)

    scores, episodes, global_step = [], [], 0

    for e in range(EPISODES):
        done = False
        dead = False

        step, score, start_life = 0, 0, 5
        observe = env.reset()

        for _ in range(random.randint(1, agent.no_op_steps)):
            observe, _, _, _ = env.step(1)

        state = pre_processing(observe)
        history = np.stack((state, state, state, state), axis=2)
        history = np.reshape([history], (1, 84, 84, 4))

        while not done:
            if agent.render:
                env.render()
            global_step += 1
            step += 1

            # 바로 전 4개의 상태로 행동을 선택
            action = agent.get_action(history)
            # 1: 정지, 2: 왼쪽, 3: 오른쪽
            if action == 0:
                real_action = 1
            elif action == 1:
                real_action = 2
            else:
                real_action = 3

            # 선택한 행동으로 환경에서 한 타임스텝 진행
            observe, reward, done, info = env.step(real_action)
            # 각 타임스텝마다 상태 전처리
            next_state = pre_processing(observe)
            next_state = np.reshape([next_state], (1, 84, 84, 1))
            next_history = np.append(next_state, history[:, :, :, :3], axis=3)

            agent.avg_q_max += np.amax(
                agent.model.predict(np.float32(history / 255.))[0])

            if start_life > info['ale.lives']:
                dead = True
                start_life = info['ale.lives']

            reward = np.clip(reward, -1., 1.)
            # 샘플 <s, a, r, s'>을 리플레이 메모리에 저장 후 학습
            agent.append_sample(history, action, reward, next_history, dead)

            if len(agent.memory) >= agent.train_start:
                agent.train_model()

            # 일정 시간마다 타겟모델을 모델의 가중치로 업데이트
            if global_step % agent.update_target_rate == 0:
                agent.update_target_model()

            score += reward

            if dead:
                dead = False
            else:
                history = next_history

            if done:
                # 각 에피소드 당 학습 정보를 기록
                if global_step > agent.train_start:
                    stats = [score, agent.avg_q_max / float(step), step,
                             agent.avg_loss / float(step)]
                    for i in range(len(stats)):
                        agent.sess.run(agent.update_ops[i], feed_dict={
                            agent.summary_placeholders[i]: float(stats[i])
                        })
                    summary_str = agent.sess.run(agent.summary_op)
                    agent.summary_writer.add_summary(summary_str, e + 1)

                print("episode:", e, "  score:", score, "  memory length:",
                      len(agent.memory), "  epsilon:", agent.epsilon,
                      "  global_step:", global_step, "  average_q:",
                      agent.avg_q_max / float(step), "  average loss:",
                      agent.avg_loss / float(step))

                agent.avg_q_max, agent.avg_loss = 0, 0

        # 1000 에피소드마다 모델 저장
        if e % 1000 == 0:
            agent.model.save_weights("./save_model/breakout_dqn.h5")


Using TensorFlow backend.
/home/ericnjin/anaconda3/envs/deep-learning-with-python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ericnjin/anaconda3/envs/deep-learning-with-python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ericnjin/anaconda3/envs/deep-learning-with-python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qi

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 20, 20, 32)        8224      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 64)          32832     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               1606144   
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 1539      
Total params: 1,685,667
Trainable params: 1,685,667
Non-trainable params: 0
____________________________________________

episode: 35   score: 2.0   memory length: 5575   epsilon: 1.0   global_step: 5575   average_q: 0.028277389962689668   average loss: 0.0
episode: 36   score: 0.0   memory length: 5677   epsilon: 1.0   global_step: 5677   average_q: 0.029275660585685103   average loss: 0.0
episode: 37   score: 1.0   memory length: 5824   epsilon: 1.0   global_step: 5824   average_q: 0.028513695461814908   average loss: 0.0
episode: 38   score: 1.0   memory length: 5980   epsilon: 1.0   global_step: 5980   average_q: 0.02876131084914773   average loss: 0.0
episode: 39   score: 1.0   memory length: 6123   epsilon: 1.0   global_step: 6123   average_q: 0.027591534779517802   average loss: 0.0
episode: 40   score: 4.0   memory length: 6392   epsilon: 1.0   global_step: 6392   average_q: 0.028619189583956088   average loss: 0.0
episode: 41   score: 3.0   memory length: 6599   epsilon: 1.0   global_step: 6599   average_q: 0.027524414755727932   average loss: 0.0
episode: 42   score: 1.0   memory length: 6753   

episode: 95   score: 0.0   memory length: 14927   epsilon: 1.0   global_step: 14927   average_q: 0.029094408636966957   average loss: 0.0
episode: 96   score: 0.0   memory length: 15050   epsilon: 1.0   global_step: 15050   average_q: 0.029411338527149302   average loss: 0.0
episode: 97   score: 1.0   memory length: 15223   epsilon: 1.0   global_step: 15223   average_q: 0.028792982178099582   average loss: 0.0
episode: 98   score: 0.0   memory length: 15333   epsilon: 1.0   global_step: 15333   average_q: 0.029251669110222298   average loss: 0.0
episode: 99   score: 0.0   memory length: 15431   epsilon: 1.0   global_step: 15431   average_q: 0.029503097225512778   average loss: 0.0
episode: 100   score: 1.0   memory length: 15597   epsilon: 1.0   global_step: 15597   average_q: 0.02892823055027479   average loss: 0.0
episode: 101   score: 0.0   memory length: 15705   epsilon: 1.0   global_step: 15705   average_q: 0.02922414144915011   average loss: 0.0
episode: 102   score: 1.0   memory

episode: 155   score: 1.0   memory length: 24467   epsilon: 1.0   global_step: 24467   average_q: 0.02770098961339701   average loss: 0.0
episode: 156   score: 1.0   memory length: 24624   epsilon: 1.0   global_step: 24624   average_q: 0.02876213841899565   average loss: 0.0
episode: 157   score: 1.0   memory length: 24777   epsilon: 1.0   global_step: 24777   average_q: 0.028402287439972747   average loss: 0.0
episode: 158   score: 4.0   memory length: 25043   epsilon: 1.0   global_step: 25043   average_q: 0.028381545327715856   average loss: 0.0
episode: 159   score: 2.0   memory length: 25232   epsilon: 1.0   global_step: 25232   average_q: 0.028842222566405933   average loss: 0.0
episode: 160   score: 0.0   memory length: 25331   epsilon: 1.0   global_step: 25331   average_q: 0.029099708046726507   average loss: 0.0
episode: 161   score: 3.0   memory length: 25568   epsilon: 1.0   global_step: 25568   average_q: 0.029402025460505285   average loss: 0.0
episode: 162   score: 1.0   m

episode: 215   score: 0.0   memory length: 34244   epsilon: 1.0   global_step: 34244   average_q: 0.029284543124958873   average loss: 0.0
episode: 216   score: 4.0   memory length: 34498   epsilon: 1.0   global_step: 34498   average_q: 0.027645418083104563   average loss: 0.0
episode: 217   score: 2.0   memory length: 34720   epsilon: 1.0   global_step: 34720   average_q: 0.029313266042385017   average loss: 0.0
episode: 218   score: 2.0   memory length: 34894   epsilon: 1.0   global_step: 34894   average_q: 0.028542264953426932   average loss: 0.0
episode: 219   score: 2.0   memory length: 35086   epsilon: 1.0   global_step: 35086   average_q: 0.02923311178650086   average loss: 0.0
episode: 220   score: 0.0   memory length: 35189   epsilon: 1.0   global_step: 35189   average_q: 0.029238161043200678   average loss: 0.0
episode: 221   score: 0.0   memory length: 35295   epsilon: 1.0   global_step: 35295   average_q: 0.02920690735907487   average loss: 0.0
episode: 222   score: 0.0   m

episode: 275   score: 2.0   memory length: 43861   epsilon: 1.0   global_step: 43861   average_q: 0.028864039809152645   average loss: 0.0
episode: 276   score: 1.0   memory length: 44020   epsilon: 1.0   global_step: 44020   average_q: 0.028372657385730894   average loss: 0.0
episode: 277   score: 2.0   memory length: 44194   epsilon: 1.0   global_step: 44194   average_q: 0.028533698123847616   average loss: 0.0
episode: 278   score: 1.0   memory length: 44359   epsilon: 1.0   global_step: 44359   average_q: 0.028859353415442237   average loss: 0.0
episode: 279   score: 2.0   memory length: 44560   epsilon: 1.0   global_step: 44560   average_q: 0.028358955584948334   average loss: 0.0
episode: 280   score: 0.0   memory length: 44680   epsilon: 1.0   global_step: 44680   average_q: 0.02915838295593858   average loss: 0.0
episode: 281   score: 1.0   memory length: 44832   epsilon: 1.0   global_step: 44832   average_q: 0.028047456357039903   average loss: 0.0
episode: 282   score: 1.0   